# Data Prep

In [ ]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip
!unzip data.zip

In [3]:
import os
import shutil

import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import tensorflow as tf

train_dir = "/kaggle/working/data/train"
test_dir = "/kaggle/working/data/test"

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Question 1

Since we have a binary classification problem, what is the best loss function for us?

* mean squared error
* binary crossentropy
* categorical crossentropy
* cosine similarity

Note: since we specify an activation for the output layer, we don't need to set from_logits=True


In [5]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',\
             optimizer=optimizers.SGD(learning_rate=0.002, momentum=0.8),\
             metrics=['acc'])

We should use binary crossentropy.

# Question 2
What's the number of parameters in the convolutional layer of our model? 

You can use the summary method for that.

* 1
* 65
* 896
* 11214912

In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 175232)            0         
                                                                 
 dense_2 (Dense)             (None, 64)                11214912  
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11215873 (42.79 MB)
Trainable params: 11215873 (42.79 MB)
Non-trainable params: 0 (0.00 Byte)
____________

# Generators and Training
For the next two questions, use the following data generator for both train and test sets:

* ImageDataGenerator(rescale=1./255)

We don't need to do any additional pre-processing for the images.
When reading the data from train/test directories, check the class_mode parameter. 

Which value should it be for a binary classification problem?

* Use batch_size=20
* Use shuffle=True for both training and test sets.
* For training use .fit() with the following params:

* model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator)


In [7]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    shuffle=True)

validation_generator = val_datagen.flow_from_directory(test_dir,
                                                        target_size=(150, 150),
                                                        batch_size=20,
                                                        class_mode='binary',
                                                        shuffle=True)

Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.


In [9]:
history = model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator)

Epoch 1/10
184/184 [==============================] - 20s 38ms/step - loss: 0.6829 - acc: 0.5572 - val_loss: 0.6465 - val_acc: 0.5697
Epoch 2/10
184/184 [==============================] - 7s 37ms/step - loss: 0.6481 - acc: 0.6111 - val_loss: 0.6327 - val_acc: 0.5632
Epoch 3/10
184/184 [==============================] - 6s 35ms/step - loss: 0.6170 - acc: 0.6587 - val_loss: 0.5840 - val_acc: 0.6754
Epoch 4/10
184/184 [==============================] - 6s 34ms/step - loss: 0.5665 - acc: 0.7142 - val_loss: 0.5618 - val_acc: 0.7146
Epoch 5/10
184/184 [==============================] - 6s 35ms/step - loss: 0.5272 - acc: 0.7452 - val_loss: 0.5476 - val_acc: 0.7059
Epoch 6/10
184/184 [==============================] - 6s 35ms/step - loss: 0.4942 - acc: 0.7743 - val_loss: 0.5124 - val_acc: 0.7516
Epoch 7/10
184/184 [==============================] - 6s 35ms/step - loss: 0.4676 - acc: 0.7903 - val_loss: 0.5053 - val_acc: 0.7669
Epoch 8/10
184/184 [==============================] - 6s 35ms/step -

# Question 3
What is the median of training accuracy for all the epochs for this model?

* 0.20
* 0.40
* 0.60
* 0.80


In [38]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [37]:
acc_median = np.median(acc)
acc_median

0.7597225904464722

# Question 4
What is the standard deviation of training loss for all the epochs for this model?

* 0.031
* 0.061
* 0.091
* 0.131

In [39]:
np.std(loss)

0.09263401092231485

# Data Augmentation
For the next two questions, we'll generate more data using data augmentations.

Add the following augmentations to your training data generator:

* rotation_range=50,
* width_shift_range=0.1,
* height_shift_range=0.1,
* zoom_range=0.1,
* horizontal_flip=True,
* fill_mode='nearest'



In [41]:
datagen = ImageDataGenerator(
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

In [42]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

In [43]:
val_datagen = ImageDataGenerator(rescale=1./255)

In [44]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150), 
                                                    batch_size=32, 
                                                    class_mode='binary')

Found 3677 images belonging to 2 classes.


In [45]:
validation_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

Found 918 images belonging to 2 classes.


# Question 5
Let's train our model for 10 more epochs using the same code as previously.

Note: make sure you don't re-create the model - we want to continue training the model we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

* 0.18
* 0.48
* 0.78
* 0.108

In [46]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator)

Epoch 1/10
115/115 [==============================] - 24s 203ms/step - loss: 0.4871 - acc: 0.7792 - val_loss: 0.4751 - val_acc: 0.7723
Epoch 2/10
115/115 [==============================] - 23s 200ms/step - loss: 0.4725 - acc: 0.7822 - val_loss: 0.5699 - val_acc: 0.7407
Epoch 3/10
115/115 [==============================] - 23s 199ms/step - loss: 0.4743 - acc: 0.7846 - val_loss: 0.4818 - val_acc: 0.7723
Epoch 4/10
115/115 [==============================] - 23s 198ms/step - loss: 0.4791 - acc: 0.7775 - val_loss: 0.4630 - val_acc: 0.7789
Epoch 5/10
115/115 [==============================] - 23s 196ms/step - loss: 0.4656 - acc: 0.7939 - val_loss: 0.5274 - val_acc: 0.7484
Epoch 6/10
115/115 [==============================] - 23s 197ms/step - loss: 0.4618 - acc: 0.7860 - val_loss: 0.4565 - val_acc: 0.7930
Epoch 7/10
115/115 [==============================] - 23s 197ms/step - loss: 0.4608 - acc: 0.7909 - val_loss: 0.5025 - val_acc: 0.7571
Epoch 8/10
115/115 [==============================] - 2

In [47]:
acc_aug = history.history['acc']
val_acc_aug = history.history['val_acc']
loss_aug = history.history['loss']
val_loss_aug = history.history['val_loss']

In [48]:
np.mean(val_loss_aug)

0.5035646766424179

# Question 6
What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

* 0.38
* 0.58
* 0.78
* 0.98


In [54]:
np.mean(val_acc_aug[6:10])

0.7619825601577759